In [ ]:
# !pip install numpy pandas matplotlib seaborn tqdm openpyxl missingno

# 목표

- 데이터별로 컬럼 확인하면서 데이터 이해하기

# Library

In [ ]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'.venv/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as msno

In [ ]:
text_data_org = pd.read_excel('./data/비식별된 해외기업별 영문 텍스트데이터.xlsx')
statistics_data_org = pd.read_excel('./data/통계청 국제표준산업분류 HSCODE 6단위 매핑.xlsx')
customs_data_org = pd.read_excel('./data/관세청_HS부호_240101.xlsx')

text_data = text_data_org.copy()
statistics_data = statistics_data_org.copy()
customs_data = customs_data_org.copy()

In [ ]:
display(text_data.head())
display(statistics_data.head())
display(customs_data.head())

# 비식별된 해외기업별 영문 텍스트데이터

In [ ]:
text_data.shape

In [ ]:
text_data.columns

In [ ]:
text_data.info()

## 기업 ID

In [ ]:
text_data['ID'].value_counts()

In [ ]:
text_data['ID'].unique()

In [ ]:
text_data['ID'] = text_data['ID'].astype(str)

In [ ]:
text_data['ID'].unique()

## 국제표준산업분류코드 (ISIC 4자리)

In [ ]:
text_data['CODE'].value_counts()

In [ ]:
text_data['CODE'].unique()

In [ ]:
print(text_data['CODE'].max(), text_data['CODE'].min())

- 세 자리는 네 자리로 바꿔줘야 함

In [ ]:
text_data['CODE'] = text_data['CODE'].apply(lambda x: f'{x:04}')

In [ ]:
text_data['CODE'].unique()

## 기업 관련 텍스트 데이터

In [ ]:
text_data['DSC'].value_counts()

In [ ]:
for idx in range(0, text_data.shape[0]):
    text = text_data.iloc[idx, 2]
    search = re.search(r'\bNone\b', text) # nan, Nan, NaN, Null, None
    if search:
        print(idx, search.group())

In [ ]:
# description 텍스트 길이 확인
text_data_DSC_cnt = text_data['DSC'].apply(lambda x: len(x.split()))
text_data_DSC_cnt.describe()

In [ ]:
# text_data_DSC_cnt[text_data_DSC_cnt == 17]
text_data.iloc[188, 2]

- 비어있는 description은 없음
- 텍스트 요약으로 비숫한 직군을 나타내는 컬럼 추가가 가능할까?
- 어떤 텍스트 요약 모델 쓸지도 생각해봐야 할 듯
- 아님 아예 GPT API로 비슷한 직군 레이블링 요청해도 괜찮을 듯
- 이 과정은 직접 검수 필요

# 관세청 HS 부호

In [ ]:
customs_data.columns

In [ ]:
customs_data.shape

In [ ]:
customs_data.info()

## 결측치 시각화

In [ ]:
msno.matrix(customs_data)
plt.gcf().set_size_inches(10, 5)
plt.show()

## HS부호

In [ ]:
customs_data['HS부호'].value_counts()

In [ ]:
customs_data['HS부호'].unique()

In [ ]:
customs_data[customs_data['HS부호'].isnull()]

In [ ]:
customs_data[customs_data['HS부호'] == ''] # ' '

In [ ]:
print(customs_data['HS부호'].max(), customs_data['HS부호'].min())

- 열자리로 바꿔줘야 함
- int to str

In [ ]:
# def zero_input(x):
#     if pd.isna(x):
#         return np.nan
#     elif len(x) == 6:
#         return '0000' + x
#     elif len(x) == 7:
#         return '000' + x
#     elif len(x) == 8:
#         return '00' + x
#     elif len(x) == 9:
#         return '0' + x
#     else:
#         return x

def zero_input(x):
    if pd.isna(x):
        return np.nan
    else:
        cnt = 10 - len(x)
        return '0' * cnt + x

customs_data['HS부호'] = customs_data['HS부호'].astype(str)
customs_data['HS부호'] = customs_data['HS부호'].apply(lambda x: zero_input(x))

In [ ]:
customs_data['HS부호'].value_counts()

In [ ]:
customs_data['HS부호'].unique()

## 한글품목명

In [ ]:
customs_data['한글품목명'].value_counts()

In [ ]:
customs_data['한글품목명'].unique()

In [ ]:
customs_data[customs_data['한글품목명'].isnull()]

In [ ]:
customs_data[customs_data['한글품목명'] == ''] # ' '

## 영문품목명

In [ ]:
customs_data['영문품목명'].value_counts()

In [ ]:
customs_data['영문품목명'].unique()

In [ ]:
customs_data[customs_data['영문품목명'].isnull()]

In [ ]:
customs_data[customs_data['영문품목명'] == ''] # ' '

## 성질통합분류코드

In [ ]:
customs_data['성질통합분류코드'].value_counts()

In [ ]:
customs_data['성질통합분류코드'].unique()

In [ ]:
customs_data[customs_data['성질통합분류코드'].isnull()]

In [ ]:
print(customs_data['성질통합분류코드'].max(), customs_data['성질통합분류코드'].min())

- int to str

In [ ]:
customs_data['성질통합분류코드'] = customs_data['성질통합분류코드'].astype(str)
customs_data['성질통합분류코드'] = customs_data['성질통합분류코드'].replace('nan', np.nan)
customs_data['성질통합분류코드'] = customs_data['성질통합분류코드'].str.replace('.0', '', regex=False)

In [ ]:
customs_data['성질통합분류코드'].value_counts()

In [ ]:
customs_data['성질통합분류코드'].unique()

## 성질통합분류코드명

In [ ]:
customs_data['성질통합분류코드명'].value_counts()

In [ ]:
customs_data['성질통합분류코드명'].unique()

In [ ]:
customs_data[customs_data['성질통합분류코드명'].isnull()]

- 괄호 보기 불편하니까 없애기

In [ ]:
text = '(디젤 2,000cc초과 승용차(2,500cc이하) - 중고차)'
res = re.sub(r'^\((.*?)\)$', r'\1', text)
print(res)

In [ ]:
def re_sub(x):
    if pd.isna(x):
        return np.nan
    else:
        return re.sub(r'^\((.*?)\)$', r'\1', x)

customs_data['성질통합분류코드명'] = customs_data['성질통합분류코드명'].apply(lambda x: re_sub(x))

In [ ]:
customs_data['성질통합분류코드명'].value_counts()

In [ ]:
customs_data['성질통합분류코드명'].unique()

- 결측치 존재하는 인덱스 구간 같은지 확인

In [ ]:
temp1 = customs_data[customs_data['성질통합분류코드'].isnull()].index.tolist()
temp2 = customs_data[customs_data['성질통합분류코드명'].isnull()].index.tolist()

print(temp1 == temp2)

# 통계청 국제표준산업분류 HSCODE 6단위 매핑

In [ ]:
statistics_data.shape

In [ ]:
statistics_data.columns

In [ ]:
statistics_data.columns = ['ISIC4_국제표준산업분류', 
                           'ISIC4_분류명', 
                           'KSIC10_한국표준산업분류',
                           'KSIC10_분류명', 
                           'HS2017_관세통계통합품목분류', 
                           'HS2017_분류명']

In [ ]:
statistics_data.info()

- Null값 존재함

## 결측치 시각화

In [ ]:
msno.matrix(statistics_data)
plt.gcf().set_size_inches(10, 5)
plt.show()

- ISIC4, KSIC10, HS2017별로 결측치가 같은 구간에 비어있어 보임

## ISIC4_국제표준산업분류

In [ ]:
statistics_data['ISIC4_국제표준산업분류'].value_counts()

In [ ]:
statistics_data['ISIC4_국제표준산업분류'].unique()

- `nan` 결측치 존재

In [ ]:
statistics_data[statistics_data['ISIC4_국제표준산업분류'].isnull()]

In [ ]:
print(statistics_data['ISIC4_국제표준산업분류'].max(), statistics_data['ISIC4_국제표준산업분류'].min())

- 세 자리는 네 자리로 바꿔줘야 함
- int to str

In [ ]:
def zero_input(x):
    if pd.isna(x):
        return np.nan
    else:
        cnt = 4 - len(x)
        return '0' * cnt + x

statistics_data['ISIC4_국제표준산업분류'] = statistics_data['ISIC4_국제표준산업분류'].astype(str)
statistics_data['ISIC4_국제표준산업분류'] = statistics_data['ISIC4_국제표준산업분류'].replace('nan', np.nan)
statistics_data['ISIC4_국제표준산업분류'] = statistics_data['ISIC4_국제표준산업분류'].str.replace('.0', '', regex=False)
statistics_data['ISIC4_국제표준산업분류'] = statistics_data['ISIC4_국제표준산업분류'].apply(lambda x: zero_input(x))

In [ ]:
statistics_data['ISIC4_국제표준산업분류'].unique()

## ISIC4_분류명

In [ ]:
statistics_data['ISIC4_분류명'].value_counts()

In [ ]:
statistics_data['ISIC4_분류명'].unique()

- 이것도 좀 업종별로 레이블링 할 수 있을까..?

In [ ]:
statistics_data[statistics_data['ISIC4_분류명'].isnull()]

- `nan` 결측치 존재
- `ISIC4_국제표준산업분류`와 겹치는 인덱스 확인

In [ ]:
temp1 = statistics_data[statistics_data['ISIC4_국제표준산업분류'].isnull()].index.tolist()
temp2 = statistics_data[statistics_data['ISIC4_분류명'].isnull()].index.tolist()

In [ ]:
in_temp1 = list(set(temp1) - set(temp2))
in_temp2 = list(set(temp2) - set(temp1))

print(temp1)
print(temp2)
print("'ISIC4_분류명'에는 없고 'ISIC4_국제표준산업분류'에만 있는 결측치 인덱스:", in_temp1)
print("'ISIC4_국제표준산업분류'에는 없고 'ISIC4_분류명'에만 있는 결측치 인덱스:", in_temp2)

In [ ]:
statistics_data.iloc[in_temp2]

- `ISIC4_분류명`에만 있는 인덱스를 확인하니 `ISIC4_국제표준산업분류`가 `5110`으로 존재함

## KSIC10_한국표준산업분류

In [ ]:
statistics_data['KSIC10_한국표준산업분류'].value_counts()

In [ ]:
statistics_data['KSIC10_한국표준산업분류'].unique()

In [ ]:
statistics_data[statistics_data['KSIC10_한국표준산업분류'].isnull()]

## KSIC10_분류명

In [ ]:
statistics_data['KSIC10_분류명'].value_counts()

In [ ]:
statistics_data['KSIC10_분류명'].unique()

- 이것도 좀 업종별로 레이블링 할 수 있을까..?

In [ ]:
statistics_data[statistics_data['KSIC10_분류명'].isnull()]

- `nan` 결측치 존재
- `KSIC10_한국표준산업분류`와 겹치는 인덱스 확인

In [ ]:
temp1 = statistics_data[statistics_data['KSIC10_한국표준산업분류'].isnull()].index.tolist()
temp2 = statistics_data[statistics_data['KSIC10_분류명'].isnull()].index.tolist()

In [ ]:
in_temp1 = list(set(temp1) - set(temp2))
in_temp2 = list(set(temp2) - set(temp1))

print(temp1)
print(temp2)
print("'KSIC10_분류명'에는 없고 'KSIC10_한국표준산업분류'에만 있는 결측치 인덱스:", in_temp1)
print("'KSIC10_한국표준산업분류'에는 없고 'KSIC10_분류명'에만 있는 결측치 인덱스:", in_temp2)

In [ ]:
statistics_data.iloc[in_temp2]

- `KSIC10_분류명`에만 있는 인덱스를 확인하니 `KSIC10_국제표준산업분류`가 `24312`으로 존재함

## HS2017_관세통계통합품목분류

In [ ]:
statistics_data['HS2017_관세통계통합품목분류'].value_counts()

In [ ]:
statistics_data['HS2017_관세통계통합품목분류'].unique()

In [ ]:
statistics_data[statistics_data['HS2017_관세통계통합품목분류'].isnull()]

- `nan` 결측치 존재

In [ ]:
print(statistics_data['HS2017_관세통계통합품목분류'].max(), statistics_data['HS2017_관세통계통합품목분류'].min())

- 여섯자리로 바꿔줘야 함
- int to str

In [ ]:
def zero_input(x):
    if pd.isna(x):
        return np.nan
    else:
        cnt = 6 - len(x)
        return '0' * cnt + x

statistics_data['HS2017_관세통계통합품목분류'] = statistics_data['HS2017_관세통계통합품목분류'].astype(str)
statistics_data['HS2017_관세통계통합품목분류'] = statistics_data['HS2017_관세통계통합품목분류'].replace('nan', np.nan)
statistics_data['HS2017_관세통계통합품목분류'] = statistics_data['HS2017_관세통계통합품목분류'].str.replace('.0', '', regex=False)
statistics_data['HS2017_관세통계통합품목분류'] = statistics_data['HS2017_관세통계통합품목분류'].apply(lambda x: zero_input(x))

In [ ]:
statistics_data['HS2017_관세통계통합품목분류'].value_counts()

In [ ]:
statistics_data['HS2017_관세통계통합품목분류'].unique()

## HS2017_분류명

In [ ]:
statistics_data['HS2017_분류명'].value_counts()

In [ ]:
statistics_data['HS2017_분류명'].unique()

In [ ]:
statistics_data[statistics_data['HS2017_분류명'].isnull()]

- 이것도 좀 업종별로 레이블링 할 수 있을까..?

- `nan` 결측치 존재
- `HS2017_관세통계통합품목분류`와 겹치는 인덱스 확인

In [ ]:
temp1 = statistics_data[statistics_data['HS2017_관세통계통합품목분류'].isnull()].index.tolist()
temp2 = statistics_data[statistics_data['HS2017_분류명'].isnull()].index.tolist()

In [ ]:
in_temp1 = list(set(temp1) - set(temp2))
in_temp2 = list(set(temp2) - set(temp1))

print(temp1)
print(temp2)
print("'HS2017_분류명'에는 없고 'HS2017_관세통계통합품목분류'에만 있는 결측치 인덱스:", in_temp1)
print("'HS2017_관세통계통합품목분류'에는 없고 'HS2017_분류명'에만 있는 결측치 인덱스:", in_temp2)

- `HS2017_관세통계통합품목분류`와 `HS2017_분류명` 둘 다 결측치가 겹침

# 데이터 다시 확인

In [ ]:
display(text_data.head())
display(statistics_data.head())
display(customs_data.head())

In [ ]:
display(text_data.info())
display(statistics_data.info())
display(customs_data.info())

- 다 object로 바꿈

In [ ]:
text_data.to_csv('./data/비식별된 해외기업별 영문 텍스트데이터.csv', index=False, encoding='utf-8')
statistics_data.to_csv('./data/통계청 국제표준산업분류 HSCODE 6단위 매핑.csv', index=False, encoding='utf-8')
customs_data.to_csv('./data/관세청_HS부호_240101.csv', index=False, encoding='utf-8')

In [ ]:
temp0 = pd.read_csv('./data/비식별된 해외기업별 영문 텍스트데이터.csv', dtype=str)
temp1 = pd.read_csv('./data/통계청 국제표준산업분류 HSCODE 6단위 매핑.csv', dtype=str)
temp2 = pd.read_csv('./data/관세청_HS부호_240101.csv', dtype=str)

In [ ]:
display(temp0.info())
display(temp1.info())
display(temp2.info())

In [ ]:
display(temp0.head())
display(temp1.head())
display(temp2.head())